In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from dateutil import parser
%matplotlib inline


from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
from sklearn.model_selection import GridSearchCV
import pickle
from lightgbm import LGBMClassifier
print('Library Loaded')

ModuleNotFoundError: No module named 'xgboost'

In [ ]:

data = './diabetes.csv'

In [ ]:
df = pd.read_csv(data)
df.shape

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:

# Correlation matrix
corrmat = df.corr()
fig = plt.figure(figsize = (12, 12))

sns.heatmap(corrmat, vmax = 1, square = True,annot=True,vmin=-1)
plt.show()

In [ ]:
df.hist(figsize=(12,12))
plt.show()

In [ ]:
sns.pairplot(df,hue='Outcome')

# Feature Engineering

In [ ]:
#check for unbalance 
df.Outcome.value_counts()

In [18]:
df.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

In [10]:
print("# rows in dataframe {0}".format(len(df)))
print("-------------------------------------------")
print("# rows missing Glucose: {0}".format(len(df.loc[df.Glucose == 0 ])))
print("# rows missing BloodPressure: {0}".format(len(df.loc[df.BloodPressure == 0 ])))
print("# rows missing SkinThickness: {0}".format(len(df.loc[df.SkinThickness == 0 ])))
print("# rows missing insulin: {0}".format(len(df.loc[df.Insulin == 0 ])))
print("# rows missing bmi: {0}".format(len(df.loc[df.BMI == 0 ])))
print("# rows missing Age: {0}".format(len(df.loc[df.Age == 0 ])))
print("# rows missing Pregnancies: {0}".format(len(df.loc[df.Pregnancies == 0 ])))
print("# rows missing DiabetesPedigreeFunction: {0}".format(len(df.loc[df.DiabetesPedigreeFunction == 0 ])))

# rows in dataframe 768
-------------------------------------------
# rows missing Glucose: 5
# rows missing BloodPressure: 35
# rows missing SkinThickness: 227
# rows missing insulin: 374
# rows missing bmi: 11
# rows missing Age: 0
# rows missing Pregnancies: 111
# rows missing DiabetesPedigreeFunction: 0


In [11]:
X = df.drop('Outcome',axis=1) # predictor feature coloumns
y = df.Outcome


X_train , X_test , y_train , y_test = train_test_split(X, y, test_size = 0.20, random_state = 10)

print('Training Set :',len(X_train))
print('Test Set :',len(X_test))
print('Training labels :',len(y_train))
print('Test Labels :',len(y_test))

Training Set : 614
Test Set : 154
Training labels : 614
Test Labels : 154


In [12]:
from sklearn.preprocessing import Imputer
#impute with mean all 0 readings

fill = Imputer(missing_values = 0 , strategy ="mean", axis=0)

X_train = fill.fit_transform(X_train)
X_test = fill.fit_transform(X_test)

ImportError: cannot import name 'Imputer' from 'sklearn.preprocessing' (C:\Users\mrina\AppData\Local\Programs\Python\Python38-32\lib\site-packages\sklearn\preprocessing\__init__.py)

# Model Building and Evaluation

In [28]:
print('Training Set :',len(X_train))
print('Test Set :',len(X_test))
print('Training labels :',len(y_train))
print('Test Labels :',len(y_test))

Training Set : 614
Test Set : 154
Training labels : 614
Test Labels : 154


In [29]:
def FitModel(X_train,y_train,X_test,y_test,algo_name,algorithm,gridSearchParams,cv):
    np.random.seed(10)
   
    
    grid = GridSearchCV(
        estimator=algorithm,
        param_grid=gridSearchParams,
        cv=cv, scoring='accuracy', verbose=1, n_jobs=-1)
    
    
    grid_result = grid.fit(X_train, y_train)
    best_params = grid_result.best_params_
    pred = grid_result.predict(X_test)
    cm = confusion_matrix(y_test, pred)
   # metrics =grid_result.gr
    print(pred)
    #pickle.dump(grid_result,open(algo_name,'wb'))
   
    print('Best Params :',best_params)
    print('Classification Report :',classification_report(y_test,pred))
    print('Accuracy Score : ' + str(accuracy_score(y_test,pred)))
    print('Confusion Matrix : \n', cm)

# Logistic Regression

In [30]:
# Create regularization penalty space
penalty = ['l1', 'l2']

# Create regularization hyperparameter space
C = np.logspace(0, 4, 10)

# Create hyperparameter options
hyperparameters = dict(C=C, penalty=penalty)

FitModel(X_train,y_train,X_test,y_test,'LogisticRegression',LogisticRegression(),hyperparameters,cv=5)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


[1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 1 0 1 0 0 0 1 0 0 1 1 0 0 0 0 1 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 1
 0 0 1 1 0 0 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 1 1 0 1 0
 1 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 1 0 0 0 0 1 0 1 1
 1 0 0 0 1 0]
Best Params : {'C': 1.0, 'penalty': 'l1'}
Classification Report :               precision    recall  f1-score   support

           0       0.75      0.91      0.82        95
           1       0.78      0.53      0.63        59

    accuracy                           0.76       154
   macro avg       0.76      0.72      0.72       154
weighted avg       0.76      0.76      0.75       154

Accuracy Score : 0.7597402597402597
Confusion Matrix : 
 [[86  9]
 [28 31]]


[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.2s finished
C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


# XgBoost

In [31]:
param ={
            'n_estimators': [100, 500, 1000,1500, 2000],
            'max_depth' :[2,3,4,5,6,7],
    'learning_rate':np.arange(0.01,0.1,0.01).tolist()
           
        }

FitModel(X_train,y_train,X_test,y_test,'XGBoost',XGBClassifier(),param,cv=5)

Fitting 5 folds for each of 270 candidates, totalling 1350 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    6.7s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   31.6s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 1226 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 1350 out of 1350 | elapsed:  3.8min finished


[1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 1 1 0 0 1 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 1 0 1 0 1 0 1 0 1 0 1 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 1 0 0 0
 0 0 1 1 0 0 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 0 0 1 1 0 1 1 0 0 1 1 1 1 0
 1 0 0 1 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 1 1 1 0 0 0 1 1 1 1 0 0 1 0 1 0 1 1
 1 1 0 0 1 0]
Best Params : {'learning_rate': 0.04, 'max_depth': 7, 'n_estimators': 100}
Classification Report :               precision    recall  f1-score   support

           0       0.80      0.83      0.81        95
           1       0.71      0.66      0.68        59

    accuracy                           0.77       154
   macro avg       0.75      0.75      0.75       154
weighted avg       0.76      0.77      0.76       154

Accuracy Score : 0.7662337662337663
Confusion Matrix : 
 [[79 16]
 [20 39]]


# Random Forest

In [32]:
param ={
            'n_estimators': [100, 500, 1000,1500, 2000],
           
        }
FitModel(X_train,y_train,X_test,y_test,'Random Forest',RandomForestClassifier(),param,cv=5)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    3.1s finished


[1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 1 1 0 0 1 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 1 0 0 0
 0 0 1 1 0 0 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 1 1 0 1 0
 1 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1 1 1 0 0 0 1 0 1 1 0 0 0 0 1 0 1 1
 1 0 0 0 1 0]
Best Params : {'n_estimators': 1500}
Classification Report :               precision    recall  f1-score   support

           0       0.75      0.86      0.80        95
           1       0.70      0.53      0.60        59

    accuracy                           0.73       154
   macro avg       0.73      0.69      0.70       154
weighted avg       0.73      0.73      0.72       154

Accuracy Score : 0.7337662337662337
Confusion Matrix : 
 [[82 13]
 [28 31]]


# SVC

In [33]:
param ={
            'C': [0.1, 1, 100, 1000],
            'gamma': [0.0001, 0.001, 0.005, 0.1, 1, 3, 5]
        }
FitModel(X_train,y_train,X_test,y_test,'SVC',SVC(),param,cv=5)

Fitting 5 folds for each of 28 candidates, totalling 140 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


[1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 1 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1
 0 0 1 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1 0
 1 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 1 0 1 0 1 0 0 0 0 0 1 1 0 0 0 0 1 0 1 1
 1 0 0 0 0 0]
Best Params : {'C': 1, 'gamma': 0.0001}
Classification Report :               precision    recall  f1-score   support

           0       0.71      0.91      0.80        95
           1       0.73      0.41      0.52        59

    accuracy                           0.71       154
   macro avg       0.72      0.66      0.66       154
weighted avg       0.72      0.71      0.69       154

Accuracy Score : 0.7142857142857143
Confusion Matrix : 
 [[86  9]
 [35 24]]


[Parallel(n_jobs=-1)]: Done 140 out of 140 | elapsed:    0.5s finished


# Balancing the Dataset - Over Sampling

In [34]:
y.value_counts()

0    500
1    268
Name: Outcome, dtype: int64

In [35]:
from imblearn.over_sampling import SMOTE
sm =SMOTE(random_state=42)
X_res_OS , Y_res_OS = sm.fit_resample(X,y)
pd.Series(Y_res_OS).value_counts()

Using TensorFlow backend.


1    500
0    500
dtype: int64

In [36]:
X_train , X_test , y_train , y_test = train_test_split(X_res_OS, Y_res_OS, test_size = 0.20, random_state = 10)

print('Training Set :',len(X_train))
print('Test Set :',len(X_test))
print('Training labels :',len(y_train))
print('Test Labels :',len(y_test))

Training Set : 800
Test Set : 200
Training labels : 800
Test Labels : 200


In [37]:

fill = Imputer(missing_values = 0 , strategy ="mean", axis=0)

X_train = fill.fit_transform(X_train)
X_test = fill.fit_transform(X_test)

C:\Users\user\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [38]:
print('Training Set :',len(X_train))
print('Test Set :',len(X_test))
print('Training labels :',len(y_train))
print('Test Labels :',len(y_test))

Training Set : 800
Test Set : 200
Training labels : 800
Test Labels : 200


# Logistic Regression - After Over sampling

In [39]:
# Create regularization penalty space
penalty = ['l1', 'l2']

# Create regularization hyperparameter space
C = np.logspace(0, 4, 10)

# Create hyperparameter options
hyperparameters = dict(C=C, penalty=penalty)

FitModel(X_train,y_train,X_test,y_test,'LogisticRegression',LogisticRegression(),hyperparameters,cv=5)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.1s finished


[1 0 0 0 1 0 0 1 1 1 0 0 1 1 0 1 1 1 1 1 1 0 0 0 0 1 1 1 1 1 0 1 0 0 0 0 0
 0 0 1 1 0 0 1 1 1 1 1 0 0 0 1 0 0 0 1 1 1 1 1 0 1 1 0 1 0 0 1 1 1 0 1 0 1
 0 1 1 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 1 0 1 1 1 1 1 0
 0 0 1 1 0 0 1 0 1 0 1 1 1 1 1 0 0 1 1 1 0 0 0 1 0 1 1 1 0 0 1 0 0 1 0 1 0
 1 1 1 1 1 0 1 1 0 1 0 0 1 1 0 1 0 0 0 1 0 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 1
 0 1 1 1 1 1 1 1 1 0 1 1 0 0 0]
Best Params : {'C': 1.0, 'penalty': 'l2'}
Classification Report :               precision    recall  f1-score   support

           0       0.68      0.72      0.70        93
           1       0.74      0.70      0.72       107

    accuracy                           0.71       200
   macro avg       0.71      0.71      0.71       200
weighted avg       0.71      0.71      0.71       200

Accuracy Score : 0.71
Confusion Matrix : 
 [[67 26]
 [32 75]]


C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


# XgBoost  - After Over sampling

In [40]:
param ={
            'n_estimators': [100, 500, 1000,1500, 2000],
            'max_depth' :[2,3,4,5,6,7],
    'learning_rate':np.arange(0.01,0.1,0.01).tolist()
           
        }

FitModel(X_train,y_train,X_test,y_test,'XGBoost',XGBClassifier(),param,cv=5)

Fitting 5 folds for each of 270 candidates, totalling 1350 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   28.9s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 1226 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 1350 out of 1350 | elapsed:  3.7min finished


[1 1 0 0 0 1 0 1 1 1 0 0 1 1 0 1 1 1 1 1 1 0 0 0 0 1 0 1 1 1 0 1 0 0 0 0 0
 0 0 1 0 1 1 1 1 1 1 1 1 0 0 1 0 0 0 1 0 1 1 1 0 1 1 0 1 0 0 1 1 1 1 0 0 1
 0 1 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 1 1 1 0 0 0 1 0 1 1 1 1 0 0
 0 0 1 0 1 1 1 1 1 0 1 0 1 1 1 0 0 1 1 1 1 0 0 1 1 1 1 1 0 0 1 1 0 1 1 1 0
 0 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 0 0 1 0 1 1 0 1 1 0 0 0 0 0 0 1 1 0 0 0
 0 1 1 1 1 1 1 1 0 0 1 1 0 0 0]
Best Params : {'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 1000}
Classification Report :               precision    recall  f1-score   support

           0       0.80      0.78      0.79        93
           1       0.82      0.83      0.82       107

    accuracy                           0.81       200
   macro avg       0.81      0.81      0.81       200
weighted avg       0.81      0.81      0.81       200

Accuracy Score : 0.81
Confusion Matrix : 
 [[73 20]
 [18 89]]


# Random Forest  - After Over sampling

In [41]:
param ={
            'n_estimators': [100, 500, 1000,1500, 2000],
           
        }
FitModel(X_train,y_train,X_test,y_test,'Random Forest',RandomForestClassifier(),param,cv=5)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


[1 1 0 0 1 1 0 1 1 1 0 0 0 1 0 1 1 1 1 1 1 0 0 0 0 1 1 1 1 1 0 1 0 0 0 0 0
 0 0 1 1 1 1 1 1 1 1 1 1 0 0 1 0 0 1 1 0 1 1 1 0 1 1 1 1 0 0 1 1 1 1 1 0 1
 0 1 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 1 0 0 0 1 0 1 1 1 1 0 0
 0 0 1 0 1 1 1 1 1 0 1 1 1 1 1 0 0 1 1 1 1 0 0 1 1 1 1 1 0 0 1 1 0 1 1 1 0
 0 1 1 1 1 1 1 1 0 1 1 0 1 1 0 1 1 0 0 1 0 1 1 0 1 1 0 0 0 0 0 0 1 1 0 0 0
 0 1 1 1 1 1 1 1 0 0 1 1 0 0 0]
Best Params : {'n_estimators': 100}
Classification Report :               precision    recall  f1-score   support

           0       0.85      0.80      0.82        93
           1       0.83      0.88      0.85       107

    accuracy                           0.84       200
   macro avg       0.84      0.84      0.84       200
weighted avg       0.84      0.84      0.84       200

Accuracy Score : 0.84
Confusion Matrix : 
 [[74 19]
 [13 94]]


[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    3.8s finished


# SVC  - After Over sampling

In [42]:
param ={
            'C': [0.1, 1, 100, 1000],
            'gamma': [0.0001, 0.001, 0.005, 0.1, 1, 3, 5]
        }
FitModel(X_train,y_train,X_test,y_test,'SVC',SVC(),param,cv=5)

Fitting 5 folds for each of 28 candidates, totalling 140 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


[1 1 0 0 0 0 0 0 1 0 0 0 1 1 0 1 1 1 1 1 1 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0
 0 0 1 1 1 1 0 1 1 1 1 0 1 0 1 1 0 1 1 0 0 1 1 0 1 1 1 1 0 0 1 1 1 0 1 0 1
 0 0 1 1 0 0 0 0 0 1 1 0 0 0 1 0 0 0 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 1 0 1 0
 0 1 1 0 1 1 1 1 1 0 1 0 0 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 0 0 1 1 0
 0 1 0 0 1 1 1 0 0 1 1 0 0 1 0 1 1 1 0 1 0 1 0 0 0 1 0 0 1 1 0 0 1 1 1 0 1
 1 1 1 1 1 1 1 1 0 0 0 1 0 0 0]
Best Params : {'C': 1, 'gamma': 0.005}
Classification Report :               precision    recall  f1-score   support

           0       0.74      0.76      0.75        93
           1       0.79      0.77      0.78       107

    accuracy                           0.77       200
   macro avg       0.76      0.76      0.76       200
weighted avg       0.77      0.77      0.77       200

Accuracy Score : 0.765
Confusion Matrix : 
 [[71 22]
 [25 82]]


[Parallel(n_jobs=-1)]: Done 140 out of 140 | elapsed:    0.8s finished
